<a href="https://colab.research.google.com/github/anisetusms/DASI_DALIS_Kelompok_06/blob/main/DASI_DALIS_Kel_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Prediksi Peluang Obesitas pada Siswa SMA/SMK di Toba dan Tapanuli Utara Berdasarkan Pola Hidup Menggunakan Machine Learning
"""
# 1. IMPORT LIBRARIES

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                            f1_score, classification_report, confusion_matrix,
                            roc_curve, roc_auc_score, auc)
from imblearn.over_sampling import SMOTE

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

print("Semua libraries berhasil dimuat!")

Semua libraries berhasil dimuat!
Semua libraries berhasil dimuat!


In [2]:
# ==========================================
# 1. LOAD DATA
# ==========================================
print("\n" + "="*70)
print("TAHAP 1: PENGUMPULAN & LOADING DATA")
print("="*70)

df = pd.read_csv("data_siswa-siswi1.csv", sep=';', encoding='latin1')
print(f" Data berhasil dimuat: {df.shape[0]} baris, {df.shape[1]} kolom")

# Tampilkan semua kolom
print("\nDaftar Kolom:")
for i, col in enumerate(df.columns):
    print(f"  {i+1}. {col}")

# Drop kolom yang completely empty
df = df.dropna(axis=1, how="all")
print(f"\n Kolom kosong dihapus: Sisa {df.shape[1]} kolom")


TAHAP 1: PENGUMPULAN & LOADING DATA
 Data berhasil dimuat: 2050 baris, 27 kolom

Daftar Kolom:
  1. Timestamp
  2. Nama lengkap
  3. Jenis Kelamin
  4. Usia
  5. Jalur Pendidikan
  6. Asal Sekolah
  7. Kelas
  8.  Berapa berat badan kamu sekarang? (dalam kilogram)  
  9. Berapa tinggi badan kamu sekarang? (dalam centimeter)
  10. Dalam 7 hari terakhir, rata-rata kamu makan utama (pagi/siang/malam) berapa kali per hari? 
  11. Dalam 7 hari terakhir, kira-kira berapa kali kamu jajan (di luar makan utama)?  
  12. Dalam 7 hari terakhir, berapa kali kamu makan fast food / makanan cepat saji
  13. Dalam 7 hari terakhir, berapa gelas/porsi minuman manis (teh manis, minuman bersoda, boba, minuman serbuk manis) yang kamu konsumsi?
  14. Secara umum, jenis makanan apa yang paling sering kamu konsumsi?  
  15. Berapa kali kamu berolahraga dalam seminggu?
  16. Jika kamu berolahraga, rata-rata berapa menit durasi tiap kali olahraga?  
  17. Seberapa sering kamu dalam melakukan aktivitas fisik se

In [3]:
# 2. DATA CLEANING & PREPROCESSING

print("\n" + "="*70)
print("TAHAP 2: PEMBERSIHAN & PREPROCESSING DATA (STANDARDIZED)")
print("="*70)

# 2.1 Fungsi untuk membersihkan angka dengan satuan
def clean_numeric(val):
    """Membersihkan nilai numerik dengan satuan (kg, cm, tahun)"""
    if pd.isna(val):
        return np.nan
    val = str(val).lower()
    for unit in ["kg", "cm", "tahun", "th", "jam", "kilogram", "centimeter"]:
        val = val.replace(unit, "")
    val = val.replace(",", ".").strip()
    try:
        return float(val)
    except:
        return np.nan



TAHAP 2: PEMBERSIHAN & PREPROCESSING DATA (STANDARDIZED)


In [5]:
# 2.2 Mapping kategori → nilai numerik (STANDARDIZED)
mapping_tidur = {"< 5 jam": 4, "5-6 jam": 5.5, "5–6 jam": 5.5, "7-8 jam": 7.5, "7–8 jam": 7.5, "> 8 jam": 9}
mapping_makan = {"1 kali": 1, "2 kali": 2, "3 kali": 3, "> 3 kali": 4}
mapping_jajan = {"0 - 2 kali": 1, "3 - 5 kali": 4, "4 - 5 kali": 4.5, "6 - 10 kali": 8, "> 10 kali": 12, "> 10  kali": 12}
mapping_fastfood = {"0 - 2 kali": 1, "3 -  5 kali": 4, "3 - 5 kali": 4, "> 5 kali": 7}
mapping_minuman = {"0 - 2 gelas": 1, "3 - 5 gelas": 4, "6 - 10 gelas": 8, "10 - 6 gelas": 8, "10 gelas": 8, ">10 gelas": 12, "> 10 gelas": 12}

mapping_makan_malam = {
    "0 Kali": 0,
    "0 kali": 0,
    "1 Kali": 1,
    "1 kali": 1,
    "2 - 3 kali": 2.5,
    "2 - 3 Kali": 2.5,
    "4 kali": 4,
    "5 Kali": 5,
    "> 4 kali": 6,
}

In [7]:
# 2.3 Bersihkan kolom numerik dasar
df["berat_kg"] = df[" Berapa berat badan kamu sekarang? (dalam kilogram)  "].apply(clean_numeric)
df["tinggi_cm"] = df["Berapa tinggi badan kamu sekarang? (dalam centimeter)"].apply(clean_numeric)
df["usia_tahun"] = df["Usia"].apply(clean_numeric)
print(" Kolom numerik (berat, tinggi, usia) dibersihkan")


 Kolom numerik (berat, tinggi, usia) dibersihkan
